In [1]:
%matplotlib inline
from tqdm import tqdm
import numpy as np
import pymc3 as pm
np.set_printoptions(suppress=True)

/Users/ded/anaconda3/lib/python3.6/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7201 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:86:00.0)
/Users/ded/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# Parameters
N = 10000 # True total individuals
S = 20  # Sample periods

# Arrival: 40% initially present
birth = np.concatenate(([.4], np.ones(S-1) * (1-.4)/(S-1)), axis=0) 

# Departure: fixed 20% rate All depart at 
dep  = np.concatenate((np.ones(S-1) * .2, [1.]), axis=0)

# Capture probability (propensity)
alpha = .25
beta = 5
p = .2

In [10]:
dep

array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
       0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1. ])

In [14]:
# Capture History Data Structures
CH = [] # Full CH matrix
GCH = [] # First/Last/Count CH matrix
TH = []

# Generate and Arrival Distribution lmd ~ Cat(arr)
ARRIVALS = pm.Categorical.dist(birth).random(size = N)

for i in tqdm(range(N)):
    arrival = ARRIVALS[i]
    dep_tmp = dep.copy()
    dep_tmp[:arrival] = 0
    dep_tmp = np.append([1],(1-dep_tmp).cumprod())[:-1] * dep_tmp # Based on the arrival, compute probs of departure
    departure = pm.Categorical.dist(dep_tmp).random() # Sample departure
    dur = departure - arrival + 1
    capture_history = pm.Bernoulli.dist(p).random(size=(dur,1))
    capture_history = np.concatenate((np.zeros(arrival), 
                                      np.array(capture_history), 
                                      np.zeros(S - departure - 1)), 
                                     axis = 0)
    TH.append([int(arrival), int(departure), dur, p, capture_history.sum()])
    if capture_history.any():
        first = capture_history.argmax()
        last = S - capture_history[::-1].argmax() -1
        count = capture_history.sum()
        GCH.append([first, last, count])
        CH.append(capture_history)

100%|██████████| 10000/10000 [03:39<00:00, 45.61it/s]


In [17]:
import pickle
fname = 'data/fixed-%d-%d-%.2f.pkl' % (N, S, p)
fname
with open(fname, 'wb') as f:
  pickle.dump((GCH, sorted(TH, key=lambda tup: tup[3])), f)
fname

'data/fixed-10000-20-0.20.pkl'

In [19]:
len(GCH)

5232